In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import regex as re

In [ ]:
data = pd.read_csv("/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Data/Wheat/wheat_final_data.csv")

In [ ]:
data.shape

(7896, 11)

In [ ]:
data.head()

,Sector,Category,Crop,QueryType,QueryText,KccAns,StateName,DistrictName,CreatedOn,CreatedOnd,month
0,agriculture,cereals,wheat,improved variety of crop,information regarding improved varieties of wh...,"VARIETIES OF WHEAT CROP:- HD 2967,PBW 621,DBW ...",PUNJAB,AMRITSAR,2011-10-03T08:49:00,2011-10-03 08:49:00.000,10.0
1,agriculture,cereals,wheat,Sowing/Harvesting_Time,information regarding sowing time of wheat?,SOWING TIME OF WHEAT:-LAST WEEK OF OCT TO LAST...,PUNJAB,AMRITSAR,2011-10-05T14:04:00,2011-10-05 14:04:00.000,10.0
2,agriculture,cereals,wheat,improved variety of crop,new varieties of wheat?,"HD-2967,PBW-621.",PUNJAB,AMRITSAR,2011-10-06T08:05:00,2011-10-06 08:05:00.000,10.0
3,agriculture,cereals,wheat,improved variety of crop,information regarding to yield of variety hd 2...,21.4 quintal per acre.,PUNJAB,AMRITSAR,2011-10-06T11:34:00,2011-10-06 11:34:00.000,10.0
4,agriculture,cereals,wheat,improved variety of crop,new varities of wheat,"PBW 621, PBW 550, HD 2967",PUNJAB,AMRITSAR,2011-10-06T16:42:00,2011-10-06 16:42:00.000,10.0


In [ ]:
new = data['QueryType'].unique()
print(new)


['improved variety of crop' 'Sowing/Harvesting_Time' 'seeds'
 'fertilizer use and availability' 'seed treatment' 'weed management'
 'cultural practices' 'plant protection' 'government schemes' 'varieties'
 'sowing time and weather' 'biopesticides and biofertilizers'
 'soil testing' 'field preparation' 'market information'
 'nutrient management' 'training and exposure visits' 'fertilizer used'
 'water management' 'crop insurance' 'agriculture mechanization'
 'post harvest preservation' 'storage' 'power roads etc' '17']


In [ ]:
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 12.0 MB 21.6 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
from tqdm import tqdm
import spacy
nlp = spacy.load("en_core_web_sm",disable=['parser','ner'])

In [ ]:
data[data['QueryType']=="improved variety of crop"]

,Sector,Category,Crop,QueryType,QueryText,KccAns,StateName,DistrictName,CreatedOn,CreatedOnd,month
0,agriculture,cereals,wheat,improved variety of crop,information regarding improved varieties of wh...,"VARIETIES OF WHEAT CROP:- HD 2967,PBW 621,DBW ...",PUNJAB,AMRITSAR,2011-10-03T08:49:00,2011-10-03 08:49:00.000,10.0
2,agriculture,cereals,wheat,improved variety of crop,new varieties of wheat?,"HD-2967,PBW-621.",PUNJAB,AMRITSAR,2011-10-06T08:05:00,2011-10-06 08:05:00.000,10.0
3,agriculture,cereals,wheat,improved variety of crop,information regarding to yield of variety hd 2...,21.4 quintal per acre.,PUNJAB,AMRITSAR,2011-10-06T11:34:00,2011-10-06 11:34:00.000,10.0
4,agriculture,cereals,wheat,improved variety of crop,new varities of wheat,"PBW 621, PBW 550, HD 2967",PUNJAB,AMRITSAR,2011-10-06T16:42:00,2011-10-06 16:42:00.000,10.0
6,agriculture,cereals,wheat,improved variety of crop,information regarding improved varieties of wh...,"VARIETIES OF WHEAT :HD 2967,PBW621,DBW17,PBW55...",PUNJAB,AMRITSAR,2011-10-11T12:26:00,2011-10-11 12:26:00.000,10.0
...,...,...,...,...,...,...,...,...,...,...,...
7748,agriculture,cereals,wheat,improved variety of crop,information regarding to the late [irrigated]v...,"THE LATE VARIETIES OF WHEAT -PBW 590,PBW 509, ...",PUNJAB,TARN TARAN,2011-12-16T17:10:00,2011-12-16 17:10:00.000,12.0
7749,agriculture,cereals,wheat,improved variety of crop,informaiton regarding of varieteis in wheat cr...,"improved varietie in wheat pbw-590,pbw-509.",PUNJAB,TARN TARAN,2011-12-23T15:52:00,2011-12-23 15:52:00.000,12.0
7754,agriculture,cereals,wheat,improved variety of crop,information regarding for thesuitable late var...,"SUITABLE LATE VARIETIES OF WHEAT IS-PBW-373,HD...",PUNJAB,TARN TARAN,2012-12-05T07:52:00,2012-12-05 07:52:00.000,12.0
7757,agriculture,cereals,wheat,improved variety of crop,varity of wheat pbw590 pbw509 pbw373,varity of wheat PBW590 PBW509 PBW373,PUNJAB,TARN TARAN,2012-12-20T12:36:00,2012-12-20 12:36:00.000,12.0


In [ ]:
from collections import Counter
import pandas as pd

queryType_list = []
top_words_list = []

for qt in new :
  data1=data[data["QueryType"]==qt]
  st = ""
  for s in tqdm(data1['QueryText']):
    st += s + ", "
  # print(st)
  nouns = [] 
  doc = nlp(st)
  for tok in doc:
    if tok.pos_=="NOUN":
      nouns.append(tok.lemma_.lower())
        # print(nouns)
  # print(nouns)
  counts = {}
  for word in nouns:
    if word not in counts:
        counts[word] = 0
    counts[word] += 1
  c = Counter(counts)
  # print(c.most_common(15))

  queryType_list.append(qt)
  top_words_list.append([row[0] for row in c.most_common(15)])
  
  
df = pd.DataFrame({"queryType": queryType_list, "top_words": top_words_list})
df.to_csv("/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/top_words.csv")

100%|██████████| 1/1 [00:00<00:00, 7345.54it/s]


In [23]:
from tqdm import tqdm
import spacy
nlp = spacy.load("en_core_web_sm",disable=['parser','ner'])

In [24]:
a = !ls '/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Lovepreet/unique'

In [25]:
a

[' wheat_unique_17.csv',
 "'wheat_unique_agriculture mechanization.csv'",
 "'wheat_unique_biopesticides and biofertilizers.csv'",
 "'wheat_unique_crop insurance.csv'",
 "'wheat_unique_cultural practices.csv'",
 "'wheat_unique_fertilizer use and availability.csv'",
 "'wheat_unique_fertilizer used.csv'",
 "'wheat_unique_field preparation.csv'",
 "'wheat_unique_government schemes.csv'",
 "'wheat_unique_improved variety of crop.csv'",
 "'wheat_unique_market information.csv'",
 "'wheat_unique_nutrient management.csv'",
 "'wheat_unique_plant protection.csv'",
 "'wheat_unique_post harvest preservation.csv'",
 "'wheat_unique_power roads etc.csv'",
 ' wheat_unique_seeds.csv',
 "'wheat_unique_seed treatment.csv'",
 "'wheat_unique_soil testing.csv'",
 ' wheat_unique_Sowing-Harvesting_Time.csv',
 "'wheat_unique_sowing time and weather.csv'",
 ' wheat_unique_storage.csv',
 "'wheat_unique_training and exposure visits.csv'",
 ' wheat_unique_varieties.csv',
 "'wheat_unique_water management.csv'",
 "'w

In [33]:
stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [34]:
def get_context(sent2,keyword):
    pattern = re.compile(f"\w\s+{keyword}\s\w+")
    prefixes_suffixes = re.findall(pattern,sent2)
    prefixes = []
    suffixes = []
    for p in prefixes_suffixes:
        l = p.split(" ")
        prefixes.append(l[0].lower())
        suffixes.append(l[-1].lower())
    prefixes = [p for p in prefixes if p not in stop_words]
    suffixes = [s for s in suffixes if s not in stop_words]
    prefixes=pd.Series(prefixes).value_counts().head(5).index
    suffixes=pd.Series(suffixes).value_counts().head(5).index
    return prefixes,suffixes

In [36]:
from collections import Counter
import pandas as pd

queryType_list = []
top_words_list = []
remark_list = []

for i in a:
  print(i)
  data1 = pd.read_csv("/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/Lovepreet/unique/{}".format(i.strip().replace("'","")))
  st = ""
  for s in tqdm(data1['ques']):
    st += s + ", "
  # print(st)
  nouns = [] 
  doc = nlp(st)
  for tok in doc:
    if tok.pos_=="NOUN":
      nouns.append(tok.lemma_.lower())
        # print(nouns)
  # print(nouns)
  counts = {}
  for word in nouns:
    if word not in counts:
        counts[word] = 0
    counts[word] += 1
  c = Counter(counts)
  # print(c.most_common(15))
  top_keywords = [row[0] for row in c.most_common(15)]
  lis = []
  for k in top_keywords:
    try:
      prefix,suffix = get_context(st,k)
      lis.append([prefix,k,suffix])
    except:
      pass

  queryType_list.append(i)
  top_words_list.append(top_keywords)
  remark_list.append(lis)
  
  
df = pd.DataFrame({"Query Type": queryType_list, "Top Words": top_words_list, "Remarks": remark_list})
df.to_csv("/content/drive/Shareddrives/SSA chatbot/SSA_Chatbot/top_words.csv",index=False)


 wheat_unique_17.csv


100%|██████████| 1/1 [00:00<00:00, 1473.75it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  del sys.path[0]


'wheat_unique_agriculture mechanization.csv'


100%|██████████| 16/16 [00:00<00:00, 75658.25it/s]


'wheat_unique_biopesticides and biofertilizers.csv'


...

100%|██████████| 137/137 [00:00<00:00, 454964.09it/s]


'wheat_unique_water management.csv'


100%|██████████| 40/40 [00:00<00:00, 138425.87it/s]


'wheat_unique_weed management.csv'


100%|██████████| 178/178 [00:00<00:00, 54400.04it/s]


In [ ]:
get_context(corpus1,"SOWING")

In [ ]:
df

,queryType,top_words
0,improved variety of crop,"[wheat, variety, information, crop, varitie, y..."
1,Sowing/Harvesting_Time,"[time, wheat, information, variety, season, so..."
2,seeds,"[wheat, seed, information, rate, variety, crop..."
3,fertilizer use and availability,"[wheat, information, crop, dose, fertilizer, d..."
4,seed treatment,"[seed, wheat, treatment, information, crop, tr..."
5,weed management,"[wheat, control, weed, information, crop, leaf..."
6,cultural practices,"[information, wheat, control, crop, variety, s..."
7,plant protection,"[wheat, information, control, crop, rust, weed..."
8,government schemes,"[information, wheat, subsidy, seed, number, co..."
9,varieties,"[wheat, information, variety, detail, crop, se..."
